# Sessões da Câmara dos Deputados 🇧🇷

[Documentação oficial](http://www.camara.leg.br/internet/plenario/result/votacao/Layout_ArquivosTXT_presencas_vota%C3%A7%C3%A3o_exportados.pdf)

**Nomes dos arquivos**

```
HEaabbcddde000000.TXT presenças - cabeçalho
LPaabbcddde000000.TXT presenças - detalhes
HEaabbcdddeffffff.TXT votos - cabeçalho
LVaabbcdddeffffff.TXT votos - detalhes
```

## Cabeçalhos

| Linha presenças | Descrição presenças                              | Linha votação | Descrição votação |
|--------------|-----------------------------------------------|---------------|-------------------|
| **aabbcddde**    | conforme a nomenclatura do cabeçalho          | **aabbcddde**     | conforme nomenclatura do header  |
| **000000**       |                                               | **ffffff**        | seqüencial da votação            |
| **dd/mm/aaaa**   | data de término da sessão                     | **dd/mm/aaaa**    | data final da votação            |
| **hh:mm:ss**     | hora de término da sessão                     | **hh:mm:ss**      | hora final da votação            |
| **xxx**          | nome do 1º presidente da Sessão (40 posições) | **xxx**           | nome do 1º presidente da Votação |
| **000**          |                                               | **zzz**           | total de votos SIM               |
| **000**          |                                               | **vvv**           | total de votos NÃO               |
| **000**          |                                               | **www**           | total de votos ABSTENÇÃO         |
| **000**          |                                               | **ggg**           | total de votos OBSTRUÇÃO         |
| **000**          |                                               | **hhh**           | total de votos BRANCO            |
| **000**          |                                               | **iii**           | total de votos do presidente     |
| **000**          |                                               | **jjj**           | total de votantes                |
| **yyy**          | total de presentes                            | **kkk**           | nome da proposição               |

In [1]:
import glob
import numpy as np
import pandas as pd

paths = glob.glob('../data/sessions/HE*')
headers = pd.DataFrame()
dtypes = {
    'header': np.str,
    'vote_number': np.str,
    'ending_date': np.str,
    'ending_time': np.str,
    'first_president': 'category',
    'yes_votes': np.int,
    'no_votes': np.int,
    'abstention_votes': np.int,
    'obstruction_votes': np.int,
    'blank_votes': np.int,
    'president_votes': np.int,
    'votes': np.int,
    'name': np.str,
}
for path in paths:
    with open(path, encoding='iso-8859-1') as file:
        attributes = file.read().split('\n')
        attributes = [attr.strip() for attr in attributes]
        row = pd.Series(dict(zip(dtypes.keys(), attributes[:13])))
        headers = headers.append(row, ignore_index=True)

Definir tipos corretos para cada uma das colunas.

In [2]:
for col, col_type in dtypes.items():
    headers[col] = headers[col].astype(col_type)

Criar uma coluna contendo data e hora do término da sessão (ou da votação, dependendo da linha).

In [3]:
headers['ending_time'] = pd.to_datetime(
    headers['ending_date'] + ' ' + headers['ending_time'], dayfirst=True)
headers.drop('ending_date', axis=1, inplace=True)

## Código do cabeçalho

**aabbcddde**

| Código | Descrição |
|--------|-----------|
| aa     | CD (sessão da Câmara) ou CC (sessão do Congresso – Câmara) ou SF (sessão do Congresso – Senado) |
| bb     | Número da Sessão legislativa (2 posições) |
| c      | O (sessão legislativa ordinária) ou E (sessão legislativa extraordinária) |
| ddd    | Número da Sessão (3 posições) |
| e      | O (sessão ordinária) ou E (sessão extraordinária) |
| ffffff | seqüencial da votação (6 posições) |

In [4]:
headers.head()

,abstention_votes,blank_votes,ending_time,first_president,header,name,no_votes,obstruction_votes,president_votes,vote_number,votes,yes_votes
0,0,0,2010-08-04 19:56:03,MARCO MAIA,CD04O169O,MPV Nº 487/2010 - PARECER DO RELATOR PELA ADM...,9,25,1,004431,231,196
1,0,0,2017-09-04 20:27:45,CARLOS MANATO,CD03O242E,345,0,0,0,000000,0,0
2,0,0,2015-06-23 20:43:45,DANIEL VILELA,CD01O165E,319,0,0,0,000000,0,0
3,3,0,2016-10-05 16:09:09,BETO MANSUR,CD02O241E,PL Nº 4567/2016 - REQUERIMENTO DE ADIAMENTO ...,262,62,1,007188,335,7
4,0,0,2007-11-12 20:41:21,MANATO,CD01O318E,366,0,0,0,000000,0,0


In [5]:
def parse_header(header, include_vote_number=False):
    attrs = pd.Series({
        'body': header[:2],
        'legislative_section_number': header[2:4],
        'legislative_schedule': header[4:5],
        'session_number': header[5:8],
        'schedule': header[8:9],
    })
    if include_vote_number:
        attrs['vote_number'] = header[9:]
    return attrs

def rename_header_categorical_variables(dataframe):
    dtypes = {
        'legislative_schedule': 'category',
        'schedule': 'category',
        'body': 'category',
    }
    for col, col_type in dtypes.items():
        dataframe[col] = dataframe[col].astype(col_type)

    categories = {
        'O': 'ordinary_session',
        'E': 'special_session',
    }

    for col in ['schedule', 'legislative_schedule']:
        dataframe[col].cat.rename_categories(categories, inplace=True)

    dataframe['body'].cat.rename_categories({
        'CD': 'chamber_of_deputies',
        'CC': 'national_congress_chamber_of_deputies',
        'SF': 'national_congress_federal_senate',
    }, inplace=True)
    
    return dataframe

def dataframe_with_head_variables(dataframe):
    new_cols = dataframe['header'].apply(parse_header)
    dataframe = pd.concat(
        [dataframe.drop('header', axis=1), new_cols], axis=1)
    dataframe = rename_header_categorical_variables(dataframe)
    
    return dataframe

In [6]:
headers = dataframe_with_head_variables(headers)

Como a variável `headers` contém dois tipos de cabeçalhos distintos - de sessões e de votações - precisamos separar para tratar cada um das informações da forma que merece.

In [7]:
import re

def is_presence_row(row):
    return not not re.match(r'\d+', row['name'])

presence_rows = headers.apply(is_presence_row, axis=1)
presence_headers = headers[presence_rows]
vote_headers = headers[~presence_rows]
del(headers)

In [8]:
vote_headers.shape[0], presence_headers.shape[0]

(2802, 2059)

In [9]:
vote_specific_cols = [
    'abstention_votes',
    'blank_votes',
    'no_votes',
    'obstruction_votes',
    'president_votes',
    'vote_number',
    'votes',
    'yes_votes',
]
presence_headers = presence_headers \
    .drop(vote_specific_cols, axis=1) \
    .rename(columns={'name': 'congresspeople_present'})
presence_headers['congresspeople_present'] = \
    presence_headers['congresspeople_present'].astype(np.int)

In [10]:
vote_headers.head()

,abstention_votes,blank_votes,ending_time,first_president,name,no_votes,obstruction_votes,president_votes,vote_number,votes,yes_votes,body,legislative_schedule,legislative_section_number,schedule,session_number
0,0,0,2010-08-04 19:56:03,MARCO MAIA,MPV Nº 487/2010 - PARECER DO RELATOR PELA ADM...,9,25,1,004431,231,196,chamber_of_deputies,ordinary_session,04,ordinary_session,169
3,3,0,2016-10-05 16:09:09,BETO MANSUR,PL Nº 4567/2016 - REQUERIMENTO DE ADIAMENTO ...,262,62,1,007188,335,7,chamber_of_deputies,ordinary_session,02,special_session,241
5,3,0,2005-07-04 21:35:44,INOCÊNCIO OLIVEIRA,MPV Nº 250/2005 - REQUERIMENTO DE RETIRADA DE...,159,72,1,002392,237,2,chamber_of_deputies,ordinary_session,03,special_session,164
6,2,0,2004-11-24 16:31:49,JOÃO PAULO CUNHA,MPV Nº 200/2004 - REQUERIMENTO DE RETIRADA DE...,250,73,2,002283,331,4,chamber_of_deputies,ordinary_session,02,special_session,257
7,6,0,2008-12-17 22:52:49,ARLINDO CHINAGLIA,PDC Nº 1345/2008 - ESCOLHA DO SR. JOSÉ JORGE ...,60,0,0,003551,336,270,chamber_of_deputies,ordinary_session,02,special_session,327


In [11]:
presence_headers.head()

,ending_time,first_president,congresspeople_present,body,legislative_schedule,legislative_section_number,schedule,session_number
1,2017-09-04 20:27:45,CARLOS MANATO,345,chamber_of_deputies,ordinary_session,03,special_session,242
2,2015-06-23 20:43:45,DANIEL VILELA,319,chamber_of_deputies,ordinary_session,01,special_session,165
4,2007-11-12 20:41:21,MANATO,366,chamber_of_deputies,ordinary_session,01,special_session,318
9,2009-02-11 20:10:40,MICHEL TEMER,479,chamber_of_deputies,ordinary_session,03,special_session,008
10,2015-09-22 02:19:44,RENAN CALHEIROS,72,national_congress_federal_senate,ordinary_session,01,special_session,021


**Sanity checks**

É esperado que o número de votos de cada votação seja igual ao somatório de todos os tipos de votos possíveis.

In [12]:
vote_cols = [col for col in vote_headers.columns if col[-6:] == '_votes']
(vote_headers['votes'] == vote_headers[vote_cols].sum(axis=1)).value_counts()

True     2794
False       8
dtype: int64

---

## Detalhes de sessões

**Conteúdo arquivo**

```
aabbcddde 000000 xxxyyy zzzwwwfff ggg
```

| Linha                | Descrição                                    |
|----------------------|----------------------------------------------|
| **aabbcddde 000000** | conforme nomenclatura do cabeçalho           |
| **xxx**              | nome do parlamentar (40 posições)            |
| **yyy**              | Presente ou <------> (ausência) – 8 posições |
| **zzz**              | sigla do partido (10 posições)               |
| **www**              | nome da UF (25 posições)                     |
| **fff**              | código do parlamentar (3 posições)           |

In [13]:
paths = glob.glob('../data/sessions/LP*')
presences = pd.DataFrame()
for path in paths:
    subset = pd.read_fwf(
        path, widths=[16, 40, 9, 10, 25, 3], header=None, encoding='iso-8859-1')
    presences = presences.append(subset)

Definir os tipos corretos para cada um dos campos.

In [14]:
dtypes = {
    'header': np.str,
    'name': 'category',
    'status': 'category',
    'party': 'category',
    'state': 'category',
    'congressperson_id': 'category',
}
presences.columns = dtypes.keys()
for col, col_type in dtypes.items():
    presences[col] = presences[col].astype(col_type)

presences['congressperson_id'] = \
    presences['congressperson_id'].apply(lambda val: str(int(val)) if val else None)

Renomear variáveis categóricas.

In [15]:
presences['status'].cat.rename_categories({
    'Presente': 'present',
    '<------>': 'absent',
}, inplace=True)

In [16]:
new_cols = presences['header'].apply(parse_header)
presences = pd.concat(
    [presences.drop('header', axis=1), new_cols], axis=1)

In [17]:
presences.head()

,name,status,party,state,congressperson_id,body,legislative_schedule,legislative_section_number,schedule,session_number
0,ALMEIDA DE JESUS,present,PL,Ceará,NaN,CD,O,01,O,078
1,ANTONIO CAMBRAIA,present,PMDB,Ceará,NaN,CD,O,01,O,078
2,EUNÍCIO OLIVEIRA,present,PMDB,Ceará,NaN,CD,O,01,O,078
3,LÉO ALCÂNTARA,present,PSDB,Ceará,NaN,CD,O,01,O,078
4,MANOEL SALVIANO,present,PSDB,Ceará,NaN,CD,O,01,O,078


---

## Detalhes de votações

**Conteúdo arquivo**

```
aabbcddde ffffff xxx yyy  www ggg hhh iii
```

| Linha votação        | Descrição votação |
|----------------------|-------------------|
| **aabbcddde ffffff** | conforme nomenclatura do header |
| **xxx**              | nome do parlamentar (40 posições) |
| **yyy**              | qualidade de voto do parlamentar (Sim, Não, Abstenção, Obstrução, Branco) – 10 posições<br>Se a votação for secreta = Presente<br>Se não participou da votação = <-------> |
| **www**              | sigla do partido (10 posições) |
| **ggg**              | nome da uf (25 posições) |
| **hhh**              | código do parlamentar (3 posições) |

In [18]:
paths = glob.glob('../data/sessions/LV*')
votes = pd.DataFrame()
for path in paths:
    subset = pd.read_fwf(
        path, widths=[16, 40, 10, 10, 25, 3], header=None, encoding='iso-8859-1')
    votes = votes.append(subset)

In [19]:
dtypes = {
    'header': np.str,
    'name': 'category',
    'vote': 'category',
    'party': 'category',
    'state': 'category',
    'congressperson_id': 'category',
}
votes.columns = dtypes.keys()
for col, col_type in dtypes.items():
    votes[col] = votes[col].astype(col_type)

votes['congressperson_id'] = \
    votes['congressperson_id'].apply(lambda val: str(int(val)) if val else None)

In [20]:
votes['vote'].cat.rename_categories({
    'Sim': 'yes',
    'Não': 'no',
    'Abstenção': 'abstention',
    'Obstrução': 'obstruction',
    'Branco': 'blank',
}, inplace=True)

In [21]:
votes = dataframe_with_head_variables(votes)

In [22]:
votes.head()

,name,vote,party,state,congressperson_id,body,legislative_schedule,legislative_section_number,schedule,session_number
0,ANGELA PORTELA,yes,PT,Roraima,1,chamber_of_deputies,ordinary_session,03,ordinary_session,244
1,FRANCISCO RODRIGUES,no,DEM,Roraima,3,chamber_of_deputies,ordinary_session,03,ordinary_session,244
2,LUCIANO CASTRO,yes,PR,Roraima,4,chamber_of_deputies,ordinary_session,03,ordinary_session,244
3,MARCIO JUNQUEIRA,no,DEM,Roraima,6,chamber_of_deputies,ordinary_session,03,ordinary_session,244
4,MARIA HELENA,yes,PSB,Roraima,5,chamber_of_deputies,ordinary_session,03,ordinary_session,244
